# MAST Rhytmic Analysis

This notebook aims at the rhytmic analysis of percussion performances.

Along its content, the notebook extracts and analyses onset information comprising student's and teacher's performances so we can then experiment with Machine Learning approaches to check useful distance metrics that better assess student's rhytmic performances.

[NOTE] For the proper execution of this beta notebook, Please ensure that the MAST rhytmic dataset is located in the "data" folder according to the following hierachy:

_PROJECT ROOT_  
 &nbsp;&nbsp;&nbsp;&nbsp; data/  
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; MAST_rhythm_subset/  
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;_ALL DATA FILES_
    


We can now start our analysis by importing all MIR necessary libraries:

In [8]:
import os
from essentia.standard import *
from essentia import Pool, array
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as ipd

For visualisation purposes, let's plot the onset representations for a specific assignments. The following code will load the audio files as monophonic signal, extract their onset times and plot this information. Such process will be repeated for two pairs of songs: a pair (reference recording, student's performance) from a sucessful assignment and another pair from a failed assignment.

First of all, let's set the necessary parameters and load onset detection required functions:

In [36]:
dataset_dir = 'data/MAST_rhythm_subset'

approved_assignment = ['51_rhy1_ref154160.wav', '51_rhy1_per154160_pass.wav']
failed_assignment = ['51_rhy1_ref166260.wav', '51_rhy1_per166260_fail.wav']

# Setting audio processing parameters
fs = 44100
dur_in_secs=30
t = np.arange(dur_in_secs*fs)/float(fs)
zero_array = t*0 #used only for plotting purposes
window_size=1024
hop_size=512

# Loading Essentia's necessary tools
od_hfc = OnsetDetection(method='hfc')
# od_complex = OnsetDetection(method='complex')
w = Windowing(type = 'hann')
fft = FFT() # this gives us a complex FFT
c2p = CartesianToPolar() # and this turns it into a pair (magnitude, phase)
onsets = Onsets()

Next, we extract the onset times for each pair (reference, performance):

In [39]:

# For the approved performance
for file in approved_assignment:
    file_name = os.path.join(dataset_dir, file)
    # Loading file as a monophonic audio
    x = MonoLoader(filename = file_name, sampleRate = fs)()
    # Let's use only the first five seconds of the signals
    x = x[:dur_in_secs*fs] 
    x = x/np.max(np.abs(x))
    
    pool = Pool()
    for frame in FrameGenerator(x, frameSize = window_size, hopSize = hop_size):
        mag, phase, = c2p(fft(w(frame)))
        pool.add('features.hfc', od_hfc(mag, phase))        

        

data/MAST_rhythm_subset/51_rhy1_ref154160.wav
[ 0.11609977  0.66176873  1.21904767  1.61378682  1.98530614  2.31038547
  3.14630389  3.44816327  3.71519279  3.95900226  4.21442175]
data/MAST_rhythm_subset/51_rhy1_per154160_pass.wav
[ 0.12770975  0.67337871  1.20743763  1.59056687  1.97369611  2.24072552
  3.16952372  3.44816327  3.70358276  3.98222232  4.34213161]
